### General Packages

In [166]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


### Load the Data

In [167]:
## Reading csv file and convert to data frame
df = pd.read_csv('Prostate_Cancer_Preprocessed.csv')


# From Scratch KNN

### Important Packages

In [168]:
from sklearn.metrics import accuracy_score

In [169]:
x_train = df.drop("diagnosis_result",axis=1)
x_train.drop(x_train.tail(30).index,inplace = True)
x_train=x_train.values        

print(x_train.shape)
# len(x_train)

(70, 8)


In [170]:

y_train =df["diagnosis_result"] # to make lables without inputs in array
y_test = y_train.tail(30).values
y_train.drop(y_train.tail(30).index,inplace = True)
y_train = y_train.values

        
y= y_train.reshape(len(y_train),1)
print(len(y))
print(y_test.shape)

70
(30,)


### Functions

In [171]:
def distance_ecq(x_traain,x_test_point):
    distances=[]
    for row in range(len(x_traain)) : #loop over rows in x_train
        current_train_piont = x_traain[row]
        # print(current_train_piont.shape)
        current_distance = 0    #initalize distance zero
        # for eacr_row
        for column in range(len(current_train_piont)): # loop over columns(feature) in each row of x_train
            current_distance += (current_train_piont[column]-x_test_point[column])**2
        # current_distance = np.subtract(current_train_piont,x_test_point)
        # acummalation = np.sum(np.square(current_distance)) 
        current_distance=np.sqrt(current_distance)
        # print(x_test_point.shape)
        distances.append(current_distance)
    # distances = pd.DataFrame(data=distances, columns=["dist"])
    return distances 

In [172]:
def nearest_neighbour(distance_point,k):
    # distance point: distance between test point and each ponit in tranining 
    # k : number of neighbours to consider for prediction
    nearest_neighbors_indices = np.argsort(distance_point)[:k]  ##index prediction
    # print(nearest_neighbors_indices)        
    return nearest_neighbors_indices
    

In [173]:
def voting (df_nearst , y_traain):
    nearest_neighbors_classes = y_traain[df_nearst]         ## prediction of index from past function
    X_flattened = nearest_neighbors_classes.flatten() ## convert from 2 dimension to 1 dimension
    # print(nearest_neighbors_classes)
    # print(X_flattened)
    prediction = np.argmax(np.bincount(X_flattened))  ## ygeeb aktr 7aga etkrrt
    # print(prediction)
    return prediction

In [174]:
def predict(predict_num):
    predict_string = []
    for i in predict_num:
        if i == 1:
            predict_string.append("B")
        else:
            predict_string.append("M")
    return predict_string

In [175]:
def knn_from_scratch(x_train , y_train ,x_test , k):
    y_pred = []
    for x_test_point in x_test:
        distance_point = distance_ecq(x_train,x_test_point)
        df_nearst_point = nearest_neighbour(distance_point, k)
        y_pred_point = voting(df_nearst_point,y_train)
        y_pred.append(y_pred_point)
        predict_string = predict(y_pred)

    return predict_string

### Implementation

In [176]:
x_test = df.tail(30).drop("diagnosis_result",axis=1).values #choose data to using in test
x_test.shape

(30, 8)

### Evaluation

In [177]:
# Fit the model with many K values
scores = []
max_score = 0
best_K = 0
for i in range(10):    
    y_pred_scratch = knn_from_scratch(x_train,y,x_test,i+1)
    accuracy = accuracy_score(y_test, y_pred_scratch)
    if accuracy > max_score:
        max_score = accuracy
        best_K = i+1
    
    scores.append( [i+1, round(accuracy*100,2)] )

c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:226: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred
c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:226: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred
c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:226: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred
c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:226: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred
c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\metri

In [178]:
print("Accuracy obtained by KNN models: ",scores)
print("Best K: ", best_K)
print("Best Accuracy: ", round(max_score*100,2))

Accuracy obtained by KNN models:  [[1, 0.0], [2, 0.0], [3, 0.0], [4, 0.0], [5, 0.0], [6, 0.0], [7, 0.0], [8, 0.0], [9, 0.0], [10, 0.0]]
Best K:  0
Best Accuracy:  0


# Sklearn KNN

### Important Packages

In [179]:
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score

### Functions

In [180]:
# Function for scaling data
def Standard_Scaler (data, col_names):
    features = data[col_names]
    scaler = StandardScaler().fit(features.values)
    features = scaler.transform(features.values)
    data[col_names] = features
    
    return data

### Implementation

In [181]:
X = df.drop( 'diagnosis_result' , axis = 1)
y = df[ ['diagnosis_result'] ]

In [182]:
X = Standard_Scaler(X, X.columns)

In [183]:
X = X.to_numpy()
y = y.to_numpy()

In [184]:
# Split the data into 80% train and 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [185]:
# Fit the model with many K values
scores=[]
max_score = 0
best_K = 0
for i in range(10):
    KNN_model = KNeighborsClassifier(n_neighbors= i+1)
    KNN_model.fit(X_train, y_train)

    # Get the accuracy of the model
    KNN_score = KNN_model.score(X_test, y_test)

    if KNN_score > max_score:
        max_score = KNN_score
        best_K = i+1
        
    scores.append([i+1, KNN_score*100])

c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

In [186]:
print("Accuracy obtained by KNN models: ",scores)
print("Best K: ", best_K)
print("Best Accuracy: ", max_score*100)

Accuracy obtained by KNN models:  [[1, 75.0], [2, 80.0], [3, 85.0], [4, 85.0], [5, 85.0], [6, 85.0], [7, 90.0], [8, 85.0], [9, 85.0], [10, 80.0]]
Best K:  7
Best Accuracy:  90.0


In [187]:
# Number of folds is 10
k_folds = KFold(n_splits = 10)

cross_model_scores = []
for i in range(10):
    KNN_model = KNeighborsClassifier(n_neighbors= i+1)

    # Get the average of the scores for the 10 models
    scores = cross_val_score(KNN_model, X, y, cv = k_folds)
    cross_model_scores.append( [i+1, round((scores.mean()*100),2)] )

c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

In [188]:
print("Cross Validation Scores: ", cross_model_scores )

Cross Validation Scores:  [[1, 70.0], [2, 71.0], [3, 83.0], [4, 81.0], [5, 84.0], [6, 81.0], [7, 81.0], [8, 80.0], [9, 83.0], [10, 84.0]]


In [189]:
X = df.drop( ['diagnosis_result', 'texture' , 'fractal_dimension'] , axis = 1)
y = df[ ['diagnosis_result'] ]

In [190]:
X = Standard_Scaler(X, X.columns)

In [191]:
X = X.to_numpy()
y = y.to_numpy()

In [192]:
# Split the data into 80% train and 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [193]:
# Fit the model with many K values
scores = []
max_score = 0
best_K = 0
for i in range(10):
    KNN_model = KNeighborsClassifier(n_neighbors= i+1)
    KNN_model.fit(X_train, y_train)

    # Get the accuracy of the model
    KNN_score = KNN_model.score(X_test, y_test)

    if KNN_score > max_score:
        max_score = KNN_score
        best_K = i+1
        
    scores.append([i+1, KNN_score*100])

c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

In [194]:
print("Accuracy obtained by KNN models: ",scores)
print("Best K: ", best_K)
print("Best Accuracy: ", max_score*100)

Accuracy obtained by KNN models:  [[1, 80.0], [2, 70.0], [3, 80.0], [4, 75.0], [5, 85.0], [6, 75.0], [7, 80.0], [8, 80.0], [9, 80.0], [10, 75.0]]
Best K:  5
Best Accuracy:  85.0


In [195]:
# Number of folds is 10
k_folds = KFold(n_splits = 10)

cross_model_scores = []
for i in range(10):
    KNN_model = KNeighborsClassifier(n_neighbors= i+1)

    # Get the average of the scores for the 10 models
    scores = cross_val_score(KNN_model, X, y, cv = k_folds)
    cross_model_scores.append( [i+1, round((scores.mean()*100),2)] )

c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\yoyow\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

In [196]:
print("Cross Validation Scores: ", cross_model_scores )

Cross Validation Scores:  [[1, 78.0], [2, 72.0], [3, 81.0], [4, 81.0], [5, 85.0], [6, 84.0], [7, 85.0], [8, 85.0], [9, 85.0], [10, 83.0]]
